In [2]:
import torch
import numpy as np
import random
import copy

In [3]:
class EA(object):
    def __init__(self,  population_size, val_loader, loss_function, input_size, reservoir_size, n_labels):
        self.population_size = population_size
        self.val_loader = val_loader
        self.loss_function = loss_function
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = n_labels

    def fitness(self, population, parents=None):
        
        # Copy paste the last results, so we don't have to calculate the loss and accuracy of an unchanged model. 
        if parents == True:
            for reservoir in population:
                reservoir['epoch'].append(reservoir['epoch'][-1]+1)
                reservoir['loss_results'].append(reservoir['loss_results'][-1])
                reservoir['accuracy_results'].append(reservoir['accuracy_results'][-1])
            
        else:
            # Evaluate the performance of every (mutated/recombinated) model in the population,
            # add the results to results list. 
            for reservoir in population:
                epoch, loss, total_accuracy = evaluation(self.val_loader, 
                                                         reservoir['model'], 
                                                         reservoir['epoch'][-1]+1, 
                                                         loss_function)
                reservoir['epoch'].append(epoch)
                reservoir['loss_results'].append(loss)
                reservoir['accuracy_results'].append(total_accuracy)

                # If we find a new best model, save it.
                # Still have to fine tune this , make a directory for all the models. 
                '''if loss < reservoir['best_loss']:
                    print('* Saving new best model *')
                    torch.save(reservoir['model'], 'trained_reservoir.model')
                    reservoir['best_loss'] = loss
                    reservoir['loss_iter'] = 0
                else:
                    reservoir['loss_iter'] += 1'''

        return population

    def mutation(self, pop, option):
        
        if option == 'random_perturbation':
            pop, mut_pop = self.random_perturbation(pop, distribution='gaussian')
        
        elif option == 'diff_mutation':
            perturb_rate = 2
            pop, mut_pop = self.diff_mutation(pop, perturb_rate)
        
        return pop, mut_pop 
    
    def diff_mutation(self, pop, perturb_rate):
        mut_pop = copy.deepcopy(pop)
        
        for reservoir in mut_pop:
            
            #Changed name to make more readable
            model = reservoir['model']
            
            # Randomly sample 2 models from the population & split them up
            sample = random.sample(pop, 2)
            sample1 = sample[0]['model']
            sample2 = sample[1]['model']
            
            # Perturb the weights
            model.W_in +=  perturb_rate * (sample1.W_in - sample2.W_in)
            model.W_r += perturb_rate * (sample1.W_r - sample2.W_r)
            model.U += perturb_rate * (sample1.U - sample2.U)
            temp_w_out = model.W_out + perturb_rate * (sample1.W_out - sample2.W_out)
            model.W_out = nn.Parameter(temp_w_out, requires_grad = False)
        
        return pop, mut_pop
    
    def random_perturbation(self, pop, distribution=None):
        mut_pop = copy.deepcopy(pop)
        
        # Using a uniform distribution to sample from
        if distribution == 'uniform':
            W_in_sample = torch.empty(self.reservoir_size, self.input_size).uniform_(-0.005, 0.005)
            W_r_sample = torch.empty(self.reservoir_size, self.reservoir_size).uniform_(-0.005, 0.005)
            W_out_sample = torch.empty(self.output_size, self.reservoir_size).uniform_(-0.005, 0.005)
            U_sample = torch.empty(self.reservoir_size, self.input_size).uniform_(-0.005, 0.005)
        
        # Using a normal distribution to sample from
        elif distribution == 'gaussian':
            W_in_sample = torch.empty(self.reservoir_size, self.input_size).normal_(0, 0.0025)
            W_r_sample = torch.empty(self.reservoir_size, self.reservoir_size).normal_(0, 0.0025)
            W_out_sample = torch.empty(self.output_size, self.reservoir_size).normal_(0, 0.0025)
            U_sample = torch.empty(self.reservoir_size, self.input_size).normal_(0, 0.0025)
        
        for reservoir in mut_pop:
            reservoir['model'].W_in += W_in_sample
            reservoir['model'].W_r += W_r_sample
            reservoir['model'].U += U_sample
            
            # We have to turn off requires grad,
            # because pytorch does not allow inplace mutations on tensors which are used for backprop.
            # See https://discuss.pytorch.org/t/leaf-variable-was-used-in-an-inplace-operation/308/2 .
            reservoir['model'].W_out.requires_grad = False
            reservoir['model'].W_out += W_out_sample
        
        return pop, mut_pop

    def recombination(self, pop):  # Idea: swapp the weight matrices among the population
        mut_pop = copy.deepcopy(pop)
        
        # Last model is not changed
        for reservoir in mut_pop:
            pass
        
        return mut_pop 

    def selection(self, pop, mut_pop, option):
        
        # Merge parents and childs
        total_pop = pop + mut_pop
        
        # Select the top performing (lowest loss)
        if option == 'loss':
            total_pop = sorted(total_pop, key=lambda k: k['loss_results'][-1]) 
            new_pop = total_pop[:len(pop)]
            
        # Select the top performing (highest accuracy)
        elif option == 'accuracy':
            total_pop = sorted(total_pop, key=lambda k: k['accuracy_results'][-1], reverse=True) 
            new_pop = total_pop[:len(pop)]
            
        
        return new_pop
    
    def step(self, pop, mutate_opt, select_opt):
        pop, mut_pop = self.mutation(pop, mutate_opt)
        
        # Fitness from non-mutated are the top validation results from previous iteration. 
        pop = self.fitness(pop, parents=True)
        
        # Fitness from mutated population are calculated 
        print('Mutation') 
        mut_pop = self.fitness(mut_pop, parents=False)
        
        #self.recombination(childs)
        
        new_pop = self.selection(pop, mut_pop, select_opt)
        
        return new_pop